In [29]:
import numpy as np
import sympy as sp

In [30]:
# Define the covariance matrix
cov_matrix = np.array([
    [0.09, -0.01, -0.03, -0.02],
    [-0.01, 0.0625, 0.02, -0.01],
    [-0.03, 0.02, 0.1225, -0.015],
    [-0.02, -0.01, -0.015, 0.0576]
])

# Define the excess returns vector
excess_returns = np.array([3.6, 3.0, 5.3, 2.7]) / 100  # Convert percentages to decimals

# Calculate the inverse of the covariance matrix
inv_cov_matrix = np.linalg.inv(cov_matrix)

# Calculate the weights of the tangency portfolio
weights = inv_cov_matrix.dot(excess_returns)
weights_normalized = weights / np.sum(weights)  # Normalize the weights to sum to 1

weights_normalized


array([0.2846145 , 0.17572449, 0.21253932, 0.32712169])

In [31]:
sp.Matrix(cov_matrix)

Matrix([
[ 0.09,  -0.01,  -0.03,  -0.02],
[-0.01, 0.0625,   0.02,  -0.01],
[-0.03,   0.02, 0.1225, -0.015],
[-0.02,  -0.01, -0.015, 0.0576]])

In [32]:
sp.Matrix(excess_returns)

Matrix([
[0.036],
[ 0.03],
[0.053],
[0.027]])

In [33]:
sp.Matrix(inv_cov_matrix.round(2))

Matrix([
[14.0,   2.0,  3.86,  6.22],
[ 2.0, 17.46, -1.97,  3.21],
[3.86, -1.97,  9.87,  3.57],
[6.22,  3.21,  3.57, 21.01]])

In [34]:
sp.Matrix(weights_normalized.round(3))

Matrix([
[0.285],
[0.176],
[0.213],
[0.327]])

In [35]:
# Expected returns including the risk-free rate (convert percentages to decimals)
full_returns = np.array([5.1, 4.5, 6.8, 4.2]) / 100

# Risk-free rate
rf = 0.015

# Calculate expected return of the tangency portfolio
expected_return_portfolio = np.dot(weights_normalized, full_returns)

# Calculate the standard deviation of the tangency portfolio
portfolio_std_dev = np.sqrt(np.dot(weights_normalized.T, np.dot(cov_matrix, weights_normalized)))

# Calculate the Sharpe ratio of the tangency portfolio
sharpe_ratio = (expected_return_portfolio - rf) / portfolio_std_dev

expected_return_portfolio, portfolio_std_dev, sharpe_ratio


(0.05061472622413245, 0.10403062960352252, 0.34234846371559896)

# ii

In [36]:
# Calculate alpha to achieve the target return using the tangency portfolio return and the risk-free rate
alpha = (0.05 - rf) / (expected_return_portfolio - rf)

# Calculate the standard deviation of this new portfolio (scaled by alpha because the risk-free asset has 0 volatility)
std_dev_adjusted = alpha * portfolio_std_dev

# Calculate the Sharpe ratio of this new portfolio
sharpe_ratio_adjusted = (0.05 - rf) / std_dev_adjusted

alpha, std_dev_adjusted, sharpe_ratio_adjusted


(0.9827395493576501, 0.10223501405595833, 0.34234846371559896)

In [37]:
# Previous weights from the tangency portfolio
tangency_weights = weights_normalized

# Calculate the new weights for the assets by scaling with alpha
new_asset_weights = alpha * tangency_weights

# Calculate the allocation to cash (risk-free asset)
cash_allocation = 1 - alpha

new_asset_weights, cash_allocation


(array([0.27970192, 0.17269141, 0.20887079, 0.32147542]), 0.017260450642349934)

In [38]:
sp.Matrix(new_asset_weights.round(3))

Matrix([
[ 0.28],
[0.173],
[0.209],
[0.321]])

# iii

In [39]:
target_std_dev = 0.29

# Calculate the scaling factor beta to achieve the target standard deviation of 29%
beta = target_std_dev / portfolio_std_dev

# Calculate the expected return for the scaled portfolio
expected_return_scaled = beta * expected_return_portfolio + (1 - beta) * rf

# Calculate the new standard deviation, which should be the target standard deviation
std_dev_scaled = beta * portfolio_std_dev  # Should be approximately 29%

# Calculate the Sharpe ratio for the scaled portfolio
sharpe_ratio_scaled = (expected_return_scaled - rf) / std_dev_scaled

beta, expected_return_scaled, std_dev_scaled, sharpe_ratio_scaled


(2.7876405353426836, 0.1142810544775237, 0.29, 0.34234846371559896)

In [40]:
# Calculate the scaled weights for each asset in the tangency portfolio
scaled_weights = beta * tangency_weights

# Calculate the total borrowing (negative cash allocation)
borrowing = np.sum(scaled_weights) - 1

scaled_weights, borrowing


(array([0.79340291, 0.48985672, 0.59248322, 0.91189769]), 1.7876405353426836)

In [41]:
sp.Matrix(scaled_weights.round(3))

Matrix([
[0.793],
[ 0.49],
[0.592],
[0.912]])

# iv

In [42]:
# Given covariance matrix (cov_matrix) and vector of ones (representing sum of weights equals to one)
ones_vector = np.ones(full_returns.shape)

# Calculate the weights for the minimum variance portfolio fully invested in assets
numerator = inv_cov_matrix @ ones_vector
denominator = ones_vector.T @ inv_cov_matrix @ ones_vector

# Minimum variance portfolio weights
weights_mvp_no_cash = numerator / denominator

weights_mvp_no_cash


array([0.27131101, 0.21541792, 0.15951476, 0.35375631])

In [43]:
sp.Matrix(weights_mvp_no_cash.round(3))

Matrix([
[0.271],
[0.215],
[ 0.16],
[0.354]])

In [44]:
# Calculate the expected return using the newly derived minimum variance weights
expected_return_mvp_no_cash = np.dot(weights_mvp_no_cash, full_returns)

# Calculate the standard deviation of the minimum variance portfolio using the newly derived weights
std_dev_mvp_no_cash = np.sqrt(np.dot(weights_mvp_no_cash.T, np.dot(cov_matrix, weights_mvp_no_cash)))

# Calculate the Sharpe ratio for the minimum variance portfolio
sharpe_ratio_mvp_no_cash = (expected_return_mvp_no_cash - rf) / std_dev_mvp_no_cash

expected_return_mvp_no_cash, std_dev_mvp_no_cash, sharpe_ratio_mvp_no_cash


(0.049235436715375794, 0.10199628596277512, 0.3356537582934193)

In [46]:
# Extract variances from the diagonal of the covariance matrix
variances = np.diag(cov_matrix)

# Calculate the standard deviations of each asset
stds = np.sqrt(variances)
sp.Matrix(stds)


Matrix([
[ 0.3],
[0.25],
[0.35],
[0.24]])